20221122
sarahfong


### separates nullomers into coding and non-coding files

using GENCODE gene annotation bed 

use pybedtools to intersect nullomers w/ GENCODE

### separate non-coding into ref chromosomes and alternative haplotype chromosomes
- drop redundant non-codingn coordinates (i.e. one locus, multiple nullomer options)


### preprocessing:
    /wynton/home/ahituv/fongsl/nullomers/bin/GENCODE/0_download_format_gencode.ipynb

    /wynton/home/ahituv/fongsl/nullomers/bin/GENCODE/0_format_mutation_file.ipynb

In [1]:
import os
import pandas as pd
import pybedtools as pbt
import subprocess
import sys

# config 

In [2]:
# append path
sys.path.append("/wynton/home/ahituv/fongsl/tools/py_/")

# import config reader
import config_readwrite as crw
import count_lines as cl

config_name = os.path.join(os.path.dirname(os.getcwd()), "config")

config, configname = crw.read_config(config_name)

In [3]:
# select config variables

GENCODE_BED = config["GENCODE"]["BED"]

NULLOMER_BED = config["DATA"]["BED"]

GENCODE_OVERLAP = config["DATA"]["GENCODE_OVERLAP"]  # write
GENCODE_NOOVERLAP = config["DATA"]["GENCODE_NOOVERLAP"]  # write
GENCODE_NOOVERLAP_REF = config["DATA"]["GENCODE_NOOVERLAP_REF"]  # write

# functions 

In [4]:
def pbtIntersection(gencode_bed, test_bed):
    
    out = test_bed.strip(".bed") + "GENCODE_overlap.bed"
    
    # make pybedtool objects
    gencode = pbt.BedTool(gencode_bed)
    test = pbt.BedTool(test_bed)
    
    # intersect -u to keep only unique loci from A
    
    test.intersect(gencode, u=True, output=out)
    

    print(cl.count_lines(out))
    print("\n\n", out)

In [8]:
def pbtSubtraction(gencode_bed, test_bed):
    
    out = test_bed.strip(".bed") + "GENCODE_no-overlap.bed"
    
    # make pybedtool objects
    gencode = pbt.BedTool(gencode_bed)
    test = pbt.BedTool(test_bed)
    
    # intersect -v to keep only the variants that DO NOT overlap (this is a subtraction. )
    # no -u for subtract. Need to assess unique values later
    
    test.intersect(gencode, v=True, output=out)
    
    print(cl.count_lines(out))
    print("\n\n", out)

# Main

# intersect mutations w/ GENCODE 

### write GENCODE_OVERLAP

In [6]:
# intersection
pbtIntersection(GENCODE_BED, NULLOMER_BED) # remove duplicates

19807


 /wynton/home/ahituv/fongsl/nullomers/data/mutationsGENCODE_overlap.bed


### write GENCODE_NOOVERLAP

In [9]:
# subtraction
pbtSubtraction(GENCODE_BED, NULLOMER_BED)

11317


 /wynton/home/ahituv/fongsl/nullomers/data/mutationsGENCODE_no-overlap.bed


## invesrtigate alternative haplotypes in the non-coding
- open non-coding file
- make list of reference chromosome names
- subset non-coding dataframe for reference chromosomes. 
- write GENCODE_NOOVERLAP_REF

In [10]:
# open the non-coding file (gencode overlap subtracted mutations)

noncoding="/wynton/home/ahituv/fongsl/nullomers/data/mutationsGENCODE_no-overlap.bed"
df = pd.read_csv(noncoding, sep='\t', header =None)
print(df.shape[0])
df.head()

11317


,0,1,2,3,4
0,chr1,897961,897962,T,C
1,chr1,897972,897973,T,G
2,chr1,1000857,1000858,G,A
3,chr1,1000857,1000858,G,T
4,chr1,1018481,1018482,A,C


In [11]:
# make a list of the reference chromosomes
from chr_functions import make_chr_list

ref_chr_list = make_chr_list()
ref_chr_list.extend(["chrX", "chrY"])

## subset non-coding reference chromosomes

In [12]:
# subset reference chromosome dataframe

ref = df.loc[df[0].isin(ref_chr_list)]
print(ref.shape[0])

9451


In [13]:
# how many mutations are on alternative haplotypes? 

df.shape[0]-ref.shape[0]

1866

1866/11317 mutations are in alternative haplotypes (17%)

In [14]:
ref.head()

,0,1,2,3,4
0,chr1,897961,897962,T,C
1,chr1,897972,897973,T,G
2,chr1,1000857,1000858,G,A
3,chr1,1000857,1000858,G,T
4,chr1,1018481,1018482,A,C


### write GENCODE_NOOVERLAP_REF

In [15]:
# save reference chromosome non-coding dataframe
ref_uniq=ref[[0,1,2]].drop_duplicates()
print(ref_uniq.shape)
ref_uniq.to_csv(GENCODE_NOOVERLAP_REF, sep='\t', header=False, index=False)

(9423, 3)
